In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.filters import gaussian

In [ ]:
def musica_image_enhancement(image, kernel=np.array([0.0625, 0.2500, 0.5000, 0.2500, 0.0625])):
    image = image.astype(float)
#     print(image.shape, np.max(image))

    levels = np.floor(np.log2(np.min((image.shape[0], image.shape[1])))).astype(int)
    kernel = np.kron(kernel, kernel.T)
    
    # Layer 01 ----------------------------------------------------------------------------------------------------
    layers_01 = dict.fromkeys(np.arange(1, levels + 1))
    
    layers_01[1] = image
    
    for i in range(2, levels + 1):
        image_temp_output = np.zeros((int(layers_01[i - 1].shape[0] / 2), int(layers_01[i - 1].shape[1] / 2), image.shape[2]))
        for j in range(0, image.shape[2]):
            smoothed_image_plane = gaussian(layers_01[i - 1][:, :, j], 25)
            image_temp_output[:, :, j] = resize(smoothed_image_plane, (int(smoothed_image_plane.shape[0] / 2), int(smoothed_image_plane.shape[1] / 2)))
        layers_01[i] = image_temp_output
#         print(layers_01[i].shape, np.max(layers_01[i]))
                
    print("<---Layer 01--->")
    
    # Layer 02 ----------------------------------------------------------------------------------------------------
    layers_02 = dict.fromkeys(np.arange(1, levels + 1))
    
    for i in range(1, levels):
        layers_02[i] = resize(layers_01[i + 1], layers_01[i].shape)
#         print(layers_02[i].shape, np.max(layers_02[i]))
    print("<---Layer 02--->")
    
    # Layer 03 ----------------------------------------------------------------------------------------------------
    layers_03 = dict.fromkeys(np.arange(1, levels + 1))
    
    for i in range(1, levels):
        layers_03[i] = layers_01[i] - layers_02[i] 
#         print(layers_03[i].shape, np.max(layers_03[i]))
        
    print("<---Layer 03--->")
     
    # Layer 04 ----------------------------------------------------------------------------------------------------
    layers_04 = dict.fromkeys(np.arange(1, levels + 1))
    
    for i in range(levels - 1, 0, -1):
        maximum = np.max(layers_03[i])
#         print("==== Iter: " + str(i) + " Max: ", np.max(layers_03[i]))
        
        a = np.power(maximum, 0.1)
#         print("==== Iter: " + str(i) + " Max^0.4: ", a)
        
        b = np.sin(np.deg2rad(layers_03[i]))
#         print("==== Iter: " + str(i) + " Max(b): ", np.max(b))
        
        c = np.power(np.absolute(layers_03[i]), 0.6)
#         print("==== Iter: " + str(i) + " Max(c): ", np.max(c))
        
        layers_04[i] = np.absolute(a * (b * c)) 
#         print(layers_04[i].shape, np.max(layers_04[i]))
        
    print("<---Layer 04--->")
     
    # Layer 05 ----------------------------------------------------------------------------------------------------
    layers_05 = dict.fromkeys(np.arange(1, levels + 1))
    
    for i in range(1, levels):
        layers_05[i] = np.floor(np.absolute(layers_01[i] + np.abs(layers_04[i])) * 255)
#         print(layers_05[i].shape, np.max(layers_05[i]), np.min(layers_05[i]))
    
    print("<---Layer 05--->")
    
    return layers_05[1]

In [ ]:
image = cv2.imread('data/chest.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.imshow(image)
plt.title('Input Image')
plt.show()

In [ ]:
image.shape

In [ ]:
plt.imshow(musica_image_enhancement(image))

In [ ]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()

# out = musica_image_enhancement(image)

# scaler.fit(out[:, : , 0])
# out[:, : , 0] = scaler.transform(out[:, : , 0]) * 255

# scaler.fit(out[:, : , 1])
# out[:, : , 1] = scaler.transform(out[:, : , 1]) * 255

# scaler.fit(out[:, : , 2])
# out[:, : , 2] = scaler.transform(out[:, : , 2]) * 255

# plt.imshow(out)